# Surface Water Monitoring Network

## Analysis

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import math
import os
import glob
import datetime
import re
#import openpyxl
#import xlrd

In [ ]:
#Read CSV files from a folder

current_directory = str(os.getcwd()) + "\\raw_data\\"
dataframes = []

all_files = []

for path, subdirs, files in os.walk(current_directory):
    for name in files:
        file_name = os.path.join(path, name)
        format_matches = [".csv"]
        exclue_matches = []
        if name not in all_files:
            if any([x in name for x in format_matches]):
                if not any([y in file_name for y in exclue_matches]):
                    try:
                        current_dataframe = pd.read_csv(file_name, low_memory=False,sep=",")
                        dataframes.append(current_dataframe)

                        pass
                    except Exception as e:
                        print("Error reading file: " + file_name)
                        print(e)
                else:
                    print("Files Excluded : " + file_name)
            else:
                print("Non Excel File: " + file_name)
        
        all_files.append(name)
all_files = []


In [ ]:
dataframes[0].head(1)

In [ ]:
#Check if dataframes have the same columns

if all([set(dataframes[0].columns) == set(df.columns) for df in dataframes]):
    print('Datasets have the same columns')
else:
    print('Datasets do not have the same columns')



In [ ]:
#Find the columns names that have found in some columns but not in others. This way we can create those columns for all the dataframes

columns = []
    
for x in range(0, len(dataframes), 1):
    for y in range(0, len(dataframes), 1):
        for z in range(0, len(dataframes[x].columns), 1):
            #print(str(z) + "||"+ str(len(dataframes[y].columns))+ "||" + str(y))
            if(dataframes[x].columns[z] in dataframes[y].columns):
                pass
            else:
                if (dataframes[x].columns[z] in columns):
                    pass
                else:
                    columns.append(dataframes[x].columns[z])
                
print(columns)

In [ ]:
#Combine all the dataframes into one

surface_water_monitoring_raw = pd.concat(dataframes)
surface_water_monitoring_raw.tail(5)

In [ ]:
#Export Combined Dataset to a CSV

surface_water_monitoring_raw.to_csv("data/surface-water-monitoring-raw.csv", sep=',',index=False,encoding='utf-8-sig')

#Shape of row data
surface_water_monitoring_raw.shape

In [ ]:
#Create a copy of the dataset

surface_water_monitoring_p1 = surface_water_monitoring_raw.copy()
surface_water_monitoring_p1.head(5)

In [ ]:
# Replace -9999 with NaN
#nb_air_quality_p1 = nb_air_quality_p1.replace(-9999,np.nan)

In [ ]:
surface_water_monitoring_p1.head(5)

In [ ]:
def find_datetime_format(dt_str):
    formats_to_check = [
        '%Y/%m/%d %I:%M:%S %p',
        '%Y-%m-%d %I:%M:%S %p',
        '%Y-%m-%d %H:%M:%S.%f %p',
        '%Y/%m/%d %I.%M.%S.%f %p',
        '%Y/%m/%d %H:%M:%S',
        '%Y-%m-%d %H:%M:%S',
        '%d/%m/%Y %I:%M:%S %p',
        '%d-%m-%Y %I:%M:%S %p',
        '%d/%m/%Y %H:%M:%S',
        '%d-%m-%Y %H:%M:%S',
        '%Y/%m/%d',
        '%Y-%m-%d',
        '%d/%m/%Y',
        '%d-%m-%Y',
    ]

    for fmt in formats_to_check:
        try:
            datetime.datetime.strptime(dt_str, fmt)
            return fmt
        except ValueError:
            pass

    return None

# Example usage
dt_str = "2003-06-10 13:00:00"
format_found = find_datetime_format(dt_str)
if format_found:
    print(f"Format found: {format_found}")
else:
    print("Format not found")


In [ ]:
#Change the date format
surface_water_monitoring_p1["RESULTDATE"] = pd.to_datetime(surface_water_monitoring_p1["RESULTDATE"],format='%Y/%m/%d %I.%M.%S.%f %p')

In [ ]:
surface_water_monitoring_p1["PARMABBR"].unique()

In [ ]:
surface_water_monitoring_p1["PARMABBR"] = surface_water_monitoring_p1["PARMABBR"].replace('NH3,Un-ion','NH3_Un-ion' , regex=True)

In [ ]:
def merge_resultid_result_depth(result_id,result_depth):
    if(result_depth == "" or math.isnan(result_depth)):
        return str(result_id).replace("-", "")
    else:
        return str(result_id).replace("-", "")+"_"+str(int(result_depth))

merge_resultid_result_depth_vec = np.vectorize(merge_resultid_result_depth)

In [ ]:
#surface_water_monitoring_p1["RESULT_ID_DEPTH"] = merge_resultid_result_depth_vec(surface_water_monitoring_p1["RESULTID"],surface_water_monitoring_p1["RESULTDEPTH"])

In [ ]:
new_analyte_columns = surface_water_monitoring_p1["RESULTDATE"].unique()
print(len(new_analyte_columns))

#### Check and remove null columns

In [ ]:
def drop_empty_columns_dataset(dataset):
    indexes = []
    for i in range(0,len(dataset.columns),1):
        if(len(dataset[dataset.columns[i]])==dataset[dataset.columns[i]].isna().sum()):
            indexes.append(dataset.columns[i])
            print(dataset.columns[i])
   
    dataset.drop(indexes,inplace=True, axis=1)
    return dataset

In [ ]:
surface_water_monitoring_p1 = drop_empty_columns_dataset(surface_water_monitoring_p1)

In [ ]:
surface_water_monitoring_p1 = surface_water_monitoring_p1.replace({None: np.nan})
#surface_water_monitoring_p1 = surface_water_monitoring_p1.replace({np.nan: ""})

In [ ]:
surface_water_monitoring_p1.shape
#surface_water_monitoring_p1.iloc[:, [3]]

In [ ]:
cols = ["RESULTID","PARMABBR","RESULTVAL","PARMFLAG","STATIONID","PARMSRCDESCE"]
#cols = ["RESULT_ID_DEPTH","PARMABBR","RESULTVAL"]
surface_water_monitoring_p1_a = surface_water_monitoring_p1[cols]

In [ ]:
def find_unique_values_in_groups(surface_water_monitoring_p1_a):
    surface_water_monitoring_groups = surface_water_monitoring_p1_a.groupby(["PARMABBR"])
    surface_water_dataframes_grouped = []
    #dataset = dataset.set_index(time_column)

    # extract keys from groups
    keys = surface_water_monitoring_groups.groups.keys()
    #indexnew = 0
    for index, x in enumerate(keys):
        #if(indexnew < 2000):
        current_dataframe = surface_water_monitoring_groups.get_group(x)
        column_name = current_dataframe["PARMABBR"].unique()[0]
        flag_name = current_dataframe["PARMABBR"].unique()[0] + "_FLAG"
        current_dataframe = current_dataframe.rename(columns={'RESULTVAL': column_name,'PARMFLAG': flag_name,'PARMSRCDESCE': column_name+"_SOURCE"})
        current_dataframe = current_dataframe.drop('PARMABBR', axis=1)
        #indexnew+=1

        cols = ["RESULTID","RESULTDATE","PARMABBR","STATIONID"]
        current_dataframe_date_data = surface_water_monitoring_p1[cols].copy()
        current_dataframe_date_data = current_dataframe_date_data[(current_dataframe_date_data["PARMABBR"]==x)]
        current_dataframe_date_data = current_dataframe_date_data.drop('STATIONID', axis=1)

        current_dataframe = pd.merge(current_dataframe, current_dataframe_date_data, how='left', left_on=["RESULTID"], right_on = ["RESULTID"])
        current_dataframe = current_dataframe.sort_values(by='RESULTDATE')
        current_dataframe["DATE_COUNT"] = current_dataframe.groupby(['RESULTDATE','STATIONID']).cumcount() + 1
        current_dataframe["DATE_COUNT"] = current_dataframe.RESULTDATE.astype(str) +"("+ current_dataframe.DATE_COUNT.astype(str) +")["+ current_dataframe.STATIONID.astype(str)+"]"

        drop_cols = ['PARMABBR','RESULTID','RESULTDATE','STATIONID']
        current_dataframe = current_dataframe.drop(drop_cols, axis=1)

        current_dataframe = current_dataframe.dropna(subset=[column_name,flag_name], how='all')
        surface_water_dataframes_grouped.append(current_dataframe)
        #current_dataframe.to_csv("data/temp/surface_water-program-"+x+".csv", sep=',',index=False,encoding='utf-8-sig')
    
    return surface_water_dataframes_grouped
        
surface_water_dataframes_grouped = find_unique_values_in_groups(surface_water_monitoring_p1_a)


In [ ]:
surface_water_dataframes_grouped[6].head(5)

In [ ]:
#Create a dataframe with unique values of datecount column 

date_count_dataframe = pd.concat(surface_water_dataframes_grouped)
surface_water_empty_df = pd.DataFrame({'DATE_COUNT':date_count_dataframe["DATE_COUNT"].unique()})

In [ ]:
# Initial dataframe, choose one of the dataframes to start the merging process
surface_water_merged_df = surface_water_empty_df.copy()

# Loop through the rest of the dataframes and merge
for df in surface_water_dataframes_grouped:
    #print(len(df))
    surface_water_merged_df = pd.merge(surface_water_merged_df, df, how='left', left_on=["DATE_COUNT"], right_on = ["DATE_COUNT"])
    

In [ ]:
surface_water_merged_df= surface_water_merged_df.drop_duplicates()

#### Create another copy of the dataset for futher pre-processing

Some methods are slow when processing data. Creating a copy of a dataset will allow us not to run the entire code during data development. 

In [ ]:
#Copy dataset to new variable

surface_water_monitoring_p2 = surface_water_merged_df.copy()

In [ ]:
surface_water_monitoring_p2.columns.to_list()

In [ ]:
surface_water_monitoring_p2.head(5)

In [ ]:
#Separate Station ID columns

def separate_data_from_date_column(raw_value,values_to_find):
    #print(raw_value)
    source_only = ""
    if(values_to_find=="RESULT_DEPTH"):
        #print(raw_value)
        source_only = re.findall('\((.*?)\)',raw_value)
    elif(values_to_find=="STATION_ID"):
        source_only = re.findall('\[(.*?)\]',raw_value)
        
    source_only = source_only[0] if len(source_only) > 0 else source_only
    #print(source_only)
    return source_only
        
separate_data_from_date_column_vec = np.vectorize(separate_data_from_date_column)

In [ ]:
surface_water_monitoring_p2["RESULT_DEPTH"] = separate_data_from_date_column_vec(surface_water_monitoring_p2["DATE_COUNT"],"RESULT_DEPTH")
surface_water_monitoring_p2["STATION_ID"] = separate_data_from_date_column_vec(surface_water_monitoring_p2["DATE_COUNT"],"STATION_ID")
surface_water_monitoring_p2["STATION_ID"] =surface_water_monitoring_p2['STATION_ID'].astype(np.int64)

In [ ]:
surface_water_monitoring_p2.head(5)

In [ ]:
#Remove data from date column

def remove_extra_data_from_date_column(raw_value):
    if(pd.isnull(raw_value)==False):
        #print(raw_value)
        value_cleaned = re.sub('\((.*?)\)','',raw_value)
        value_cleaned = re.sub('\[(.*?)\]','',value_cleaned)
        #print(value_cleaned)
        return value_cleaned
    else:
        return ""  
        
remove_extra_data_from_date_column_vec = np.vectorize(remove_extra_data_from_date_column)

In [ ]:
surface_water_monitoring_p2["DATE_COUNT"] = remove_extra_data_from_date_column_vec(surface_water_monitoring_p2["DATE_COUNT"])

In [ ]:
#Change date format

surface_water_monitoring_p2["DATE_COUNT"] = pd.to_datetime(surface_water_monitoring_p2["DATE_COUNT"],format='%Y-%m-%d %H:%M:%S')

#Create Year Column

surface_water_monitoring_p2["YEAR"] = surface_water_monitoring_p2["DATE_COUNT"].dt.year

#Rename columns before station informtion
surface_water_monitoring_p2.rename(columns={'DATE_COUNT': 'DATE_TIME'}, inplace=True)

surface_water_monitoring_p2.head(5)

#### Station information

In [ ]:
#Import data

station_information = pd.read_csv("data/surface-water-monitoring-stations.csv")

#station_information["MOST_RECENT_SAMPLE_DATE"] = pd.to_datetime(station_information["MOST_RECENT_SAMPLE_DATE"],format='%Y/%m/%d %I.%M.%S.%f00000000 %p')

station_information["DMS_LATITUDE"] = np.nan
station_information["DMS_LONGITUDE"] = np.nan

In [ ]:
# Convert DMS (degrees, minutes, seconds) to DD (decimal degrees)
def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60)
    if direction == 'S' or direction == 'W':
        dd *= -1
    return dd

def dd2dms(dms,dd, pre_fix_latlong):
    if(pd.isnull(dms) == True and pd.isnull(dd) == False):
        d = int(dd)
        md = abs(dd - d) * 60
        m = int(md)
        sd = (md - m) * 60
        #return [d, m, sd] 
        #print("%s %s˚ %s' %s\"" % (pre_fix_latlong,abs(d),m,round(sd,1)))
        return "%s %s˚ %s' %s\"" % (pre_fix_latlong,abs(d),m,round(sd,1))
    return dms

def parse_dms(dms,latlong):
    if(pd.isnull(dms) != True):
        #print(dms)
        dms=dms.replace('"','')
        degDirection, minutes, seconds = re.split('[˚\']', dms)
        direction,deg = re.split('[\s]', degDirection)
        #print(deg, minutes, seconds, direction)
        latLng = dms2dd(deg, minutes, seconds, direction)

        return (latLng)
    else:
        return latlong

#dd = parse_dms("36°57'9' N 110°4'21' W")

#print(parse_dms("W 67˚ 44' 01.3",np.nan))


In [ ]:
station_information["DMS_LATITUDE"] = station_information.apply(lambda x: dd2dms(x["DMS_LATITUDE"],x["LATITUDE"],"N"),axis=1)
station_information["DMS_LONGITUDE"] = station_information.apply(lambda x: dd2dms(x["DMS_LONGITUDE"],x["LONGITUDE"],"W"),axis=1)

station_information["LATITUDE"] = station_information.apply(lambda x: parse_dms(x["DMS_LATITUDE"],x["LATITUDE"]),axis=1)
station_information["LONGITUDE"] = station_information.apply(lambda x: parse_dms(x["DMS_LONGITUDE"],x["LONGITUDE"]),axis=1)

station_information = station_information.drop_duplicates()

#Export stations to a CSV

station_information.to_csv("data/surface-water-monitoring-stations.csv", sep=',',index=False,encoding='utf-8-sig')


In [ ]:
#Attach station information

surface_water_monitoring_p2 = pd.merge(surface_water_monitoring_p2, station_information[["STATION_ID","STATION_NAME","LATITUDE","LONGITUDE"]],  how='left', left_on=['STATION_ID'], right_on = ['STATION_ID'])

surface_water_monitoring_p2.head(5)

In [ ]:
#surface_water_monitoring_p2 = surface_water_monitoring_p2.convert_dtypes()
#surface_water_monitoring_p2["RESULTVAL"] = surface_water_monitoring_p2["RESULTVAL"].astype("string", errors="ignore")
#surface_water_monitoring_p2["DO"] = surface_water_monitoring_p2["DO"].str.strip()
#surface_water_monitoring_p2 = surface_water_monitoring_p2.replace({"": np.nan})
#surface_water_monitoring_p2= surface_water_monitoring_p2.fillna(np.nan)
#surface_water_monitoring_p2.head(5)

In [ ]:
# Replace -9999 with NaN
#nb_surface_water_monitoring_p1 = nb_surface_water_monitoring_p1.replace(-9999,np.nan)


In [ ]:
#Check for null values
pd.set_option('display.max_rows',None)
#pd.set_option('display.max_columns', None)
surface_water_monitoring_p2.isna().sum()

In [ ]:
pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')

In [ ]:
surface_water_monitoring_p2 = drop_empty_columns_dataset(surface_water_monitoring_p2) #finish this later when we have whole data

In [ ]:
print(surface_water_monitoring_p2.columns.tolist())

##### Remove unit information field value where there is no analyte value

In [ ]:
def remove_unit_from_empty(unitVal, unitName):
    if(unitVal == "" or math.isnan(unitVal)):
        return np.nan
    return unitName


In [ ]:
""" nb_surface_water_monitoring_p2["SO2_INFO"] = nb_surface_water_monitoring_p2.apply(lambda x: remove_unit_from_empty(x["SO2"],x["SO2_INFO"]),axis=1) """

In [ ]:
#recreational_beach_monitoring_p2.columns.tolist()

In [ ]:
def clean_column_names(column_name):
    unit_only = re.findall('\((.*?)\)',column_name)
    unit_only = unit_only[0] if len(unit_only) > 0 else unit_only
    column_name_cleaned = re.sub('\((.*?)\)','',column_name)
    column_name_cleaned = column_name_cleaned.replace(" - ", "-")
    column_name_cleaned = column_name_cleaned.replace("  ", "_")
    column_name_cleaned = column_name_cleaned.replace(" ", "_")
    column_name_cleaned = column_name_cleaned.replace("-", "_")
    column_name_cleaned = column_name_cleaned.replace(".", "_")
    column_name_cleaned = column_name_cleaned.replace(",", "_")
    column_name_cleaned = column_name_cleaned.upper()
    return [column_name_cleaned, unit_only]

In [ ]:
#Rename all columns

surface_water_monitoring_p2 = surface_water_monitoring_p2.rename(columns=lambda x: clean_column_names(x)[0])

In [ ]:
# Get units from columns and store in a dataframe

surface_water_monitoring_units = surface_water_monitoring_p1.copy()

cols = ["PARMCD","PARMABBR","PARMDESCE","PARMDESCF","UNITDESCE","UNITDESCF"]

surface_water_monitoring_units = surface_water_monitoring_units[cols]

surface_water_monitoring_units = surface_water_monitoring_units.drop_duplicates()

surface_water_monitoring_units = surface_water_monitoring_units.rename(columns={"PARMCD":"UNIT_ID","PARMABBR":"UNIT_NAME","PARMDESCE":"UNIT_DESC_EN","PARMDESCF":"UNIT_DESC_FR","UNITDESCE":"UNIT","UNITDESCF":"UNIT_FR"})

surface_water_monitoring_units = surface_water_monitoring_units.dropna(subset = ["UNIT_NAME"])

#Export Combined Dataset to a CSV

surface_water_monitoring_units.to_csv("data/surface-water-monitoring-units.csv", sep=',',index=False,encoding='utf-8-sig')

surface_water_monitoring_units.head(5)

In [ ]:
# Get flags from columns and store in a dataframe
surface_water_monitoring_flags = surface_water_monitoring_p1.copy()

cols = ["PARMFLAG","PARMFLAG_DESC_E","PARMFLAG_DESC_F"]

surface_water_monitoring_flags = surface_water_monitoring_flags[cols]

surface_water_monitoring_flags = surface_water_monitoring_flags.drop_duplicates()

surface_water_monitoring_flags = surface_water_monitoring_flags.rename(columns={"PARMFLAG":"FLAG_VALUE","PARMFLAG_DESC_E":"FLAG_DESC_EN","PARMFLAG_DESC_F":"FLAG_DESC_FR"})

surface_water_monitoring_flags = surface_water_monitoring_flags.dropna(subset = ["FLAG_VALUE"])
#Export Combined Dataset to a CSV

surface_water_monitoring_flags.to_csv("data/surface-water-monitoring-flags.csv", sep=',',index=False,encoding='utf-8-sig')

surface_water_monitoring_flags.head(5)

In [ ]:
#Manually rename calculated variables

surface_water_monitoring_p2 = surface_water_monitoring_p2.rename(columns={"Þ_=TDS":"TDS_CALC","Þ_=TDS_SOURCE":"TDS_CALC_SOURCE"})
 

In [ ]:
surface_water_monitoring_p2.columns.tolist()

In [ ]:
#surface_water_monitoring_p2 = surface_water_monitoring_p2[(surface_water_monitoring_p2['RESULTDEPTH'].isnull() == False)]

#surface_water_monitoring_p2["DATE_TIME"].value_counts()

In [ ]:
#Round the Coulmns to 1 decimal point

#cols = ['AL_ENV_LAB', 'ALK_G_ENV_LAB']

#recreational_beach_monitoring_p2[cols] = recreational_beach_monitoring_p2[cols].round(2)

In [ ]:
#surface_water_monitoring_p2 = surface_water_monitoring_p2.astype("string", errors="ignore")
surface_water_monitoring_p2.info(verbose=True)

In [ ]:
#Create a copy of data

surface_water_monitoring_p3 = surface_water_monitoring_p2.copy()

In [ ]:
#Change the datatype for columns

surface_water_monitoring_p3["LATITUDE"] =surface_water_monitoring_p3['LATITUDE'].astype(str)
surface_water_monitoring_p3["LONGITUDE"] =surface_water_monitoring_p3['LONGITUDE'].astype(str)
surface_water_monitoring_p3["RESULT_DEPTH"] =surface_water_monitoring_p3['RESULT_DEPTH'].astype(np.int64)

In [ ]:
#Manually rename columns with only one source value

#surface_water_monitoring_p3 = surface_water_monitoring_p3.rename(columns={"ALK_G":"ALK_G_ENV_LAB"})

##### Create a final copy of processed data

In [ ]:
surface_water_monitoring = surface_water_monitoring_p2.copy()

In [ ]:
#cols = ['STATION_ID','DATE_TIME', 'ALK_G', 'ALK_G_FLAG', 'ALK_T', 'ALK_T_FLAG', 'AG', 'AG_FLAG', 'AIR_TEMP', 'AL', 'AS', 'AS_FLAG', 'B', 'BE_X', 'BE_X_FLAG', 'BR2', 'BR2_FLAG', 'BA', 'BI', 'BI_FLAG', 'CLRA', 'CLRA_FLAG', 'CLRT', 'CLRT_FLAG', 'COND', 'COND_FLAG', 'CA', 'CA_FLAG', 'CD', 'CD_FLAG', 'CL', 'CL_FLAG', 'CO', 'CO_FLAG', 'CR', 'CR_FLAG', 'CU', 'CU_FLAG', 'DO', 'DOC', 'E_COLI_MPN', 'E_COLI_MPN_FLAG', 'F', 'F_FLAG', 'FE', 'FE_FLAG', 'HARD', 'HARD_FLAG', 'K', 'LI', 'MG', 'MN', 'MN_FLAG', 'MO', 'MO_FLAG', 'NH3T', 'NH3T_FLAG', 'NH3_UN_ION', 'NH3_UN_ION_FLAG', 'NO2', 'NO2_FLAG', 'NO3', 'NO3_FLAG', 'NA', 'NI', 'NI_FLAG', 'PB', 'PB_FLAG', 'RB', 'SO4', 'SO4_FLAG', 'SS', 'SS_FLAG', 'SALINITY', 'SB', 'SB_FLAG', 'SE', 'SE_FLAG', 'SN', 'SN_FLAG', 'SR', 'TC_MPN', 'TC_MPN_FLAG', 'TEMP', 'TKN', 'TKN_FLAG', 'TN', 'TN_FLAG', 'TOC', 'TOC_FLAG', 'TP_L', 'TP_L_FLAG', 'TURB', 'TURB_FLAG', 'TE', 'TE_FLAG', 'TL', 'TL_FLAG', 'U', 'U_FLAG', 'V', 'V_FLAG', 'ZN', 'ZN_FLAG', 'PH', 'PH_FLAG', 'TDS_CALC']

#surface_water_monitoring[cols] = surface_water_monitoring[cols].replace(np.nan,"")

In [ ]:
#Empty String to NaN

#cols = ['STATION_ID','RESULT_ID_DEPTH', 'DATE_TIME', 'YEAR', 'STATION_NAME', 'LATITUDE', 'LONGITUDE', 'ALK_G', 'ALK_T', 'AG', 'AG_FLAG', 'AL', 'AS', 'AS_FLAG', 'B', 'B_FLAG', 'BE_X', 'BE_X_FLAG', 'BR2', 'BR2_FLAG', 'BA', 'BA_FLAG', 'BI', 'BI_FLAG', 'BR', 'BR_FLAG', 'CHL_A', 'CHL_A_FLAG', 'CLRA', 'CLRA_FLAG', 'CLRT', 'CLRT_FLAG', 'COND', 'CA', 'CD', 'CD_FLAG', 'CL', 'CO', 'CO_FLAG', 'CR', 'CR_FLAG', 'CU', 'CU_FLAG', 'DO', 'DOC', 'E_COLI_MPN', 'E_COLI_MPN_FLAG', 'F', 'F_FLAG', 'FE', 'FE_FLAG', 'HARD', 'HG', 'HG_FLAG', 'K', 'LI', 'MG', 'MN', 'MN_FLAG', 'MO', 'MO_FLAG', 'NH3T', 'NH3T_FLAG', 'NH3_UN_ION', 'NH3_UN_ION_FLAG', 'NO2', 'NO2_FLAG', 'NO3', 'NO3_FLAG', 'NA', 'NI', 'NI_FLAG', 'PB', 'PB_FLAG', 'RB', 'SO4', 'SO4_FLAG', 'SS', 'SS_FLAG', 'SB', 'SB_FLAG', 'SE', 'SE_FLAG', 'SECCHIDEP', 'SN', 'SN_FLAG', 'SR', 'TC_MPN', 'TEMP', 'TKN', 'TKN_FLAG', 'TN', 'TN_FLAG', 'TOC', 'TOC_FLAG', 'TP_L', 'TP_L_FLAG', 'TURB', 'TURB_FLAG', 'TE', 'TE_FLAG', 'TL', 'TL_FLAG', 'U', 'U_FLAG', 'V', 'V_FLAG', 'ZN', 'ZN_FLAG', 'PH', 'TDS_CALC']

#surface_water_monitoring_p3[cols] = surface_water_monitoring_p3[cols].replace("",np.nan)

In [ ]:
#print(surface_water_monitoring_p3['E_COLI_MPN_FLAG'].unique())

In [ ]:
#Drop null columns
surface_water_monitoring = drop_empty_columns_dataset(surface_water_monitoring)

In [ ]:
#Drop duplicate columns 
#cols = ['STATION_ID','RESULT_ID_DEPTH', 'DATE_TIME', 'YEAR', 'STATION_NAME', 'LATITUDE', 'LONGITUDE', 'ALK_G', 'ALK_T', 'AG', 'AG_FLAG', 'AL', 'AS', 'AS_FLAG', 'B', 'B_FLAG', 'BE_X', 'BE_X_FLAG', 'BR2', 'BR2_FLAG', 'BA', 'BA_FLAG', 'BI', 'BI_FLAG', 'BR', 'BR_FLAG', 'CHL_A', 'CHL_A_FLAG', 'CLRA', 'CLRA_FLAG', 'CLRT', 'CLRT_FLAG', 'COND', 'CA', 'CD', 'CD_FLAG', 'CL', 'CO', 'CO_FLAG', 'CR', 'CR_FLAG', 'CU', 'CU_FLAG', 'DO', 'DOC', 'E_COLI_MPN', 'E_COLI_MPN_FLAG', 'F', 'F_FLAG', 'FE', 'FE_FLAG', 'HARD', 'HG', 'HG_FLAG', 'K', 'LI', 'MG', 'MN', 'MN_FLAG', 'MO', 'MO_FLAG', 'NH3T', 'NH3T_FLAG', 'NH3_UN_ION', 'NH3_UN_ION_FLAG', 'NO2', 'NO2_FLAG', 'NO3', 'NO3_FLAG', 'NA', 'NI', 'NI_FLAG', 'PB', 'PB_FLAG', 'RB', 'SO4', 'SO4_FLAG', 'SS', 'SS_FLAG', 'SB', 'SB_FLAG', 'SE', 'SE_FLAG', 'SECCHIDEP', 'SN', 'SN_FLAG', 'SR', 'TC_MPN', 'TEMP', 'TKN', 'TKN_FLAG', 'TN', 'TN_FLAG', 'TOC', 'TOC_FLAG', 'TP_L', 'TP_L_FLAG', 'TURB', 'TURB_FLAG', 'TE', 'TE_FLAG', 'TL', 'TL_FLAG', 'U', 'U_FLAG', 'V', 'V_FLAG', 'ZN', 'ZN_FLAG', 'PH', 'TDS_CALC']

#recreational_beach_monitoring_p4 = recreational_beach_monitoring_p4.drop(cols, axis=1)

In [ ]:
#Rearrange columns

cols = ['STATION_ID', 'STATION_NAME', 'DATE_TIME', 'LATITUDE', 'LONGITUDE', 'YEAR', 'ALK_G', 'ALK_G_FLAG', 'ALK_G_SOURCE', 'ALK_T', 'ALK_T_FLAG', 'ALK_T_SOURCE', 'AG', 'AG_FLAG', 'AG_SOURCE', 'AIR_TEMP', 'AIR_TEMP_SOURCE', 'AL', 'AL_SOURCE', 'AS', 'AS_FLAG', 'AS_SOURCE', 'B', 'B_SOURCE', 'BE_X', 'BE_X_FLAG', 'BE_X_SOURCE', 'BR2', 'BR2_FLAG', 'BR2_SOURCE', 'BA', 'BA_SOURCE', 'BI', 'BI_FLAG', 'BI_SOURCE', 'CLRA', 'CLRA_FLAG', 'CLRA_SOURCE', 'CLRT', 'CLRT_FLAG', 'CLRT_SOURCE', 'COND', 'COND_FLAG', 'COND_SOURCE', 'CA', 'CA_FLAG', 'CA_SOURCE', 'CD', 'CD_FLAG', 'CD_SOURCE', 'CL', 'CL_FLAG', 'CL_SOURCE', 'CO', 'CO_FLAG', 'CO_SOURCE', 'CR', 'CR_FLAG', 'CR_SOURCE', 'CU', 'CU_FLAG', 'CU_SOURCE', 'DO', 'DO_SOURCE', 'DOC', 'DOC_SOURCE', 'E_COLI_MPN', 'E_COLI_MPN_FLAG', 'E_COLI_MPN_SOURCE', 'F', 'F_FLAG', 'F_SOURCE', 'FE', 'FE_FLAG', 'FE_SOURCE', 'HARD', 'HARD_FLAG', 'HARD_SOURCE', 'K', 'K_SOURCE', 'LI', 'LI_SOURCE', 'MG', 'MG_SOURCE', 'MN', 'MN_FLAG', 'MN_SOURCE', 'MO', 'MO_FLAG', 'MO_SOURCE', 'NH3T', 'NH3T_FLAG', 'NH3T_SOURCE', 'NH3_UN_ION', 'NH3_UN_ION_FLAG', 'NH3_UN_ION_SOURCE', 'NO2', 'NO2_FLAG', 'NO2_SOURCE', 'NO3', 'NO3_FLAG', 'NO3_SOURCE', 'NA', 'NA_SOURCE', 'NI', 'NI_FLAG', 'NI_SOURCE', 'PB', 'PB_FLAG', 'PB_SOURCE', 'RB', 'RB_SOURCE', 'SO4', 'SO4_FLAG', 'SO4_SOURCE', 'SS', 'SS_FLAG', 'SS_SOURCE', 'SALINITY', 'SALINITY_SOURCE', 'SB', 'SB_FLAG', 'SB_SOURCE', 'SE', 'SE_FLAG', 'SE_SOURCE', 'SN', 'SN_FLAG', 'SN_SOURCE', 'SR', 'SR_SOURCE', 'TC_MPN', 'TC_MPN_FLAG', 'TC_MPN_SOURCE', 'TEMP', 'TEMP_SOURCE', 'TKN', 'TKN_FLAG', 'TKN_SOURCE', 'TN', 'TN_FLAG', 'TN_SOURCE', 'TOC', 'TOC_FLAG', 'TOC_SOURCE', 'TP_L', 'TP_L_FLAG', 'TP_L_SOURCE', 'TURB', 'TURB_FLAG', 'TURB_SOURCE', 'TE', 'TE_FLAG', 'TE_SOURCE', 'TL', 'TL_FLAG', 'TL_SOURCE', 'U', 'U_FLAG', 'U_SOURCE', 'V', 'V_FLAG', 'V_SOURCE', 'ZN', 'ZN_FLAG', 'ZN_SOURCE', 'PH', 'PH_FLAG', 'PH_SOURCE', 'TDS_CALC', 'TDS_CALC_SOURCE']

surface_water_monitoring = surface_water_monitoring[cols]

In [ ]:
#Export Combined Dataset to a CSV

surface_water_monitoring.to_csv("data/surface-water-monitoring.csv", sep=',',index=False,encoding='utf-8-sig')

#Shape of row data
surface_water_monitoring.shape

In [ ]:
surface_water_monitoring.info(verbose=True)

In [ ]:
surface_water_monitoring.columns.tolist()

#### Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### Null Values 

In [ ]:

#Plot null values
def plot_null_values(dataset,group_by,year_filter_switch, year_filter):
    if year_filter_switch:
        dataset = dataset[(dataset['YEAR'] == year_filter)]
    
    dataset = dataset.groupby([group_by])
    # extract keys from groups
    keys = dataset.groups.keys()

    totalCols=1
    totalRows=math.ceil(len(dataset)/totalCols)
    
    fig = plt.figure(figsize=((totalCols+3)*4,(totalRows+1)*5))
    plt.subplots_adjust(wspace=0.2, hspace=0.7)

    for index, x in enumerate(keys):
        null_columns = []
        null_column_values = []

        #print(dataset.get_group(x).columns[2])

        for i in range(0,len(dataset.get_group(x).columns),1):
            if(dataset[dataset.get_group(x).columns[i]].get_group(x).isna().sum() > 0):
                null_columns.append(dataset.get_group(x).columns[i])
                null_column_values.append(dataset[dataset.get_group(x).columns[i]].get_group(x).isna().sum())

        globals()[f"ax_count_plots_{index}"] = fig.add_subplot(totalRows,totalCols,(index+1))
        globals()[f"ax_count_plots_{index}"].set_title(x.upper(),backgroundcolor='gray')
        

        globals()[f"ax_count_plots_{index}"].bar(null_columns,null_column_values)
        
        globals()[f"ax_count_plots_{index}"].set(xlabel=None)
        globals()[f"ax_count_plots_{index}"].tick_params(axis='x', labelrotation = 90)
        
    plt.show()


In [ ]:
#plot_null_values(surface_water_monitoring,'STATION_NAME',False,2020) #false if dont want to use year filter

In [ ]:
#Check for null values
pd.set_option('display.max_rows',None)
#pd.set_option('display.max_columns', None)
surface_water_monitoring.isna().sum()

In [ ]:
pd.reset_option('display.max_rows')

#### Analyse variables 

In [ ]:

#This is just an example of a line graph, visualization can be better
def linechart_of_categories(dataset,group_by,time_column,value_column,year_filter_switch,year_filter):
    if year_filter_switch:
        dataset = dataset[(dataset['YEAR'] == year_filter)]

    dataset = dataset.set_index(time_column)
    dataset = dataset.groupby([group_by])
    # extract keys from groups
    keys = dataset.groups.keys()
    totalRows = 0
    for index, x in enumerate(keys):
        if(len(dataset[value_column].get_group(x))!=dataset[value_column].get_group(x).isna().sum()):
            totalRows+=1
    
    totalCols=3
    totalRows=math.ceil(totalRows/totalCols)
    
    fig = plt.figure(figsize=((totalCols+3)*3,(totalRows+1)*5))
    plt.subplots_adjust(wspace=0.2, hspace=0.6)
    newInx = 1
    for index, x in enumerate(keys):
        if(len(dataset[value_column].get_group(x))!=dataset[value_column].get_group(x).isna().sum()):
            globals()[f"ax_count_plots_{index}"] = fig.add_subplot(totalRows,totalCols,newInx)
            globals()[f"ax_count_plots_{index}"].set_title(x.upper())
            #if(len(dataset[value_column].get_group(x))!=dataset[value_column].get_group(x).isna().sum()):
            dataset[value_column].get_group(x).plot()
            
            globals()[f"ax_count_plots_{index}"].set(xlabel=None)
            globals()[f"ax_count_plots_{index}"].tick_params(axis='x', labelrotation = 90)
            newInx+=1
    plt.show()


In [ ]:
#linechart_of_categories(surface_water_monitoring,'STATION_NAME','DATE_TIME','ALK_G',False,2022)

In [ ]:
#Display unique values

def unique_values__or_count(listOfColumns,options,dataset):
    for x in range(0, len(listOfColumns), 1):
        if(options=="unique"):
            unique_values_str = dataset[listOfColumns[x]].unique()
            print("unique_values " + listOfColumns[x])
            print(unique_values_str)
            print("------------------------")
        if(options=="count"):
            values_distribution = dataset[listOfColumns[x]].value_counts()
            print("-----------"+listOfColumns[x] +"------------")
            print(values_distribution)
            print("-----------------------")

In [ ]:
#Check station values

unique_values__or_count(['STATION_NAME'],"unique",surface_water_monitoring)


#### Data Validation

In [ ]:
#Import cleaned data 

surface_water_monitoring_validate = pd.read_csv("data/surface-water-monitoring.csv", low_memory=False)

In [ ]:
surface_water_monitoring_validate.columns.tolist()

In [ ]:
Col_name_to_validate = "MN_FLAG"

In [ ]:
surface_water_monitoring_validate["DATE_TIME"] = pd.to_datetime(surface_water_monitoring_validate["DATE_TIME"],format='%Y-%m-%d %H:%M:%S')
surface_water_monitoring_validate["DATE_TIME"] = pd.to_datetime(surface_water_monitoring_validate["DATE_TIME"].dt.strftime('%Y-%m-%d'))
surface_water_monitoring_validate = surface_water_monitoring_validate[["STATION_NAME", "DATE_TIME", Col_name_to_validate]].copy()

In [ ]:
surface_water_monitoring_validate = surface_water_monitoring_validate.dropna(subset = [Col_name_to_validate])

In [ ]:
surface_water_monitoring_validate.info(verbose=True)

In [ ]:
#surface_water_monitoring_validate["STATION_ID"] =surface_water_monitoring_validate['STATION_ID'].astype(str)

In [ ]:
surface_water_monitoring_validate.head(5)

In [ ]:
#Import original data 

surface_water_monitoring_original = pd.read_csv("data/nb_surface_water_monitoring_check.csv", low_memory=False, sep=",")


In [ ]:
print("List of Columns")
print(surface_water_monitoring_original.columns.to_list())
print("---------------")

In [ ]:
col_to_validate_with = "MN_FLAG"
#surface_water_monitoring_original = surface_water_monitoring_original[(surface_water_monitoring_original['PARMABBR'] == col_to_validate_with)]

In [ ]:
#Rename columns 
#surface_water_monitoring_original.rename(columns={'STATIONID': 'STATION_ID', 'RESULTDATE': 'DATE_TIME',"RESULTID":"RESULT_ID","RESULTVAL":Col_name_to_validate}, inplace=True)
surface_water_monitoring_original.rename(columns={'DATE': 'DATE_TIME',col_to_validate_with:Col_name_to_validate}, inplace=True)

#rakes_program_original = rakes_program_original.rename(columns=lambda x: clean_column_names(x)[0])

In [ ]:
#Change date format
#surface_water_monitoring_original["DATE_TIME"] = pd.to_datetime(surface_water_monitoring_original["DATE_TIME"],format='%Y/%m/%d %I.%M.%S.%f %p')
surface_water_monitoring_original["DATE_TIME"] = pd.to_datetime(surface_water_monitoring_original["DATE_TIME"],format='%Y/%m/%d')
surface_water_monitoring_original["DATE_TIME"] = pd.to_datetime(surface_water_monitoring_original["DATE_TIME"].dt.strftime('%Y-%m-%d'))

#Trim data to validate an analyte
surface_water_monitoring_original = surface_water_monitoring_original[["STATION_NAME", "DATE_TIME", Col_name_to_validate]].copy()

In [ ]:
#surface_water_monitoring_validate.info()
surface_water_monitoring_original = surface_water_monitoring_original.dropna(subset = [Col_name_to_validate])

In [ ]:
surface_water_monitoring_original.info()

In [ ]:
surface_water_monitoring_original.to_csv("data/temp-1.csv", sep=',',index=False,encoding='utf-8-sig')

In [ ]:
surface_water_monitoring_validate.to_csv("data/temp-2.csv", sep=',',index=False,encoding='utf-8-sig')

In [ ]:
surface_water_monitoring_validate_results = pd.merge(surface_water_monitoring_validate, surface_water_monitoring_original, on=["STATION_NAME", "DATE_TIME",Col_name_to_validate], how='right', indicator='Exist')
surface_water_monitoring_validate_results['Exist'] = np.where(surface_water_monitoring_validate_results.Exist == 'both', True, False)

In [ ]:
unique_values__or_count(['Exist'],"count",surface_water_monitoring_validate_results)

In [ ]:
list_of_missing_rows = surface_water_monitoring_validate_results[(surface_water_monitoring_validate_results['Exist'] == False)].copy()

list_of_missing_rows.head(5)

#surface_water_monitoring_validate_results.to_csv("data/temp.csv", sep=',',index=False,encoding='utf-8-sig')